In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# === Helper Function: process and average by country-year ===
def process_monthly_climate(filepath, variable_prefix, lookup_gdf):
    df = pd.read_csv(filepath)
    df['geometry'] = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

    gdf_proj = gdf.to_crs("EPSG:3857")
    lookup_proj = lookup_gdf.to_crs("EPSG:3857")
    joined = gpd.sjoin_nearest(gdf_proj, lookup_proj, how='left', distance_col='distance')
    joined = joined.to_crs("EPSG:4326").drop(columns='geometry')

    month_cols = [col for col in df.columns if 'month' in col.lower()]
    new_col_map = {col: f"{variable_prefix}_{i+1}" for i, col in enumerate(month_cols)}
    joined = joined.rename(columns=new_col_map)

    monthly_avg = joined.groupby(['country', 'year'])[[*new_col_map.values()]].mean().reset_index()
    return monthly_avg

# === Load lookup and yield data ===
lookup_df = pd.read_csv("/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/country_latitude_longitude_area_lookup.csv")
lookup_df['geometry'] = [Point(xy) for xy in zip(lookup_df['longitude'], lookup_df['latitude'])]
lookup_gdf = gpd.GeoDataFrame(lookup_df, geometry='geometry', crs='EPSG:4326')

yield_df = pd.read_csv("/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/Yield_and_Production_data.csv")
yield_df.columns = [c.strip().lower() for c in yield_df.columns]
yield_df = yield_df[yield_df["element"] == "Yield"]
if "yield" not in yield_df.columns and "value" in yield_df.columns:
    yield_df = yield_df.rename(columns={"value": "yield"})

# === Define all climate datasets (including CanopInt) ===
climate_files = {
    "rain": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/Rainf_tavg_data.csv",
    "snow": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/Snowf_tavg_data.csv",
    "esoil": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/ESoil_tavg_data.csv",
    "soilmoisture_0_10": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilMoi0_10cm_inst_data.csv",
    "soilmoisture_10_40": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilMoi10_40cm_inst_data.csv",
    "soilmoisture_40_100": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilMoi40_100cm_inst_data.csv",
    "soilmoisture_100_200": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilMoi100_200cm_inst_data.csv",
    "soiltemp_0_10": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilTMP0_10cm_inst_data.csv",
    "soiltemp_10_40": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilTMP10_40cm_inst_data.csv",
    "soiltemp_40_100": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilTMP40_100cm_inst_data.csv",
    "soiltemp_100_200": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/SoilTMP100_200cm_inst_data.csv",
    "tveg": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/TVeg_tavg_data.csv",
    "tws": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/TWS_inst_data.csv",
    "canopint": "/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/CanopInt_inst_data.csv"  # Newly added dataset
}

# === Process and merge all climate datasets ===
merged_climate = None
for prefix, path in climate_files.items():
    climate_df = process_monthly_climate(path, prefix, lookup_gdf)
    if merged_climate is None:
        merged_climate = climate_df
    else:
        merged_climate = pd.merge(merged_climate, climate_df, on=["country", "year"], how="outer")

# === Process Land Cover Percent (annual, per country) ===
land_cover_df = pd.read_csv("/Users/suli/Documents/source/repo/MachineLearning/Final_Assignment/ML 2025 Coursework Dataset - A1/Land_Cover_Percent_data.csv")
land_cover_df['geometry'] = [Point(xy) for xy in zip(land_cover_df['longitude'], land_cover_df['latitude'])]
land_cover_gdf = gpd.GeoDataFrame(land_cover_df, geometry='geometry', crs='EPSG:4326')

land_cov_joined = gpd.sjoin_nearest(land_cover_gdf.to_crs("EPSG:3857"), lookup_gdf.to_crs("EPSG:3857"), how='left')
land_cov_joined = land_cov_joined.to_crs("EPSG:4326").drop(columns='geometry')

land_class_cols = [col for col in land_cover_df.columns if "class_" in col.lower()]
land_cov_col_map = {col: f"mean_cov_{col.split('_')[-1]}" for col in land_class_cols}
land_cov_joined = land_cov_joined.rename(columns=land_cov_col_map)

land_cov_summary = land_cov_joined.groupby("country")[list(land_cov_col_map.values())].mean().reset_index()

# === Merge with climate on country + year ===
climate_with_land = pd.merge(merged_climate, land_cov_summary, on="country", how="left")

# === Merge with yield data ===
final_df = pd.merge(yield_df, climate_with_land, on=["country", "year"], how="inner")

# === Organize column order ===
def ordered_columns(final_df):
    rain_cols = [f"rain_{i}" for i in range(1, 13)]
    snow_cols = [f"snow_{i}" for i in range(1, 13)]
    esoil_cols = [f"esoil_{i}" for i in range(1, 13)]
    soilm_cols, soilt_cols = [], []
    for layer in ["0_10", "10_40", "40_100", "100_200"]:
        soilm_cols += [f"soilmoisture_{layer}_{i}" for i in range(1, 13)]
        soilt_cols += [f"soiltemp_{layer}_{i}" for i in range(1, 13)]

    tveg_cols = [f"tveg_{i}" for i in range(1, 13)]
    tws_cols = [f"tws_{i}" for i in range(1, 13)]
    canopint_cols = [f"canopint_{i}" for i in range(1, 13)]
    landcov_cols = [f"mean_cov_{i}" for i in range(1, 18)]

    base_cols = ["country", "item", "year"]
    all_features = landcov_cols + rain_cols + snow_cols + esoil_cols + soilm_cols + soilt_cols + tveg_cols + tws_cols + canopint_cols
    all_features = [col for col in all_features if col in final_df.columns]

    return base_cols + all_features + ["yield"]

final_df = final_df[ordered_columns(final_df)]

# === Save final output ===
final_df.to_csv("V5_final_with_canopint.csv", index=False)

KeyboardInterrupt: 